In [1]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug('Needed to run for Jupyter Notebook')

DEBUG:root:Needed to run for Jupyter Notebook


In [2]:
import acme

In [3]:
environment_library = 'gym'

In [4]:
import IPython

from acme import environment_loop
from acme import specs
from acme import wrappers
from acme.agents.tf import d4pg
from acme.tf import networks
from acme.tf import utils as tf2_utils
from acme.utils import loggers
import numpy as np
import sonnet as snt

from absl import app
from absl import flags
import acme
from acme.agents.tf import dqn
from acme.tf import networks

# Import the selected environment lib
if environment_library == 'dm_control':
  from dm_control import suite
elif environment_library == 'gym':
  import gym

# Imports required for visualization
import pyvirtualdisplay
import imageio
import base64

# Set up a virtual display for rendering.
display = pyvirtualdisplay.Display(visible=0, size=(1400, 900)).start()

DEBUG:matplotlib:matplotlib data path: /home/hayato/anaconda3/envs/acme/lib/python3.8/site-packages/matplotlib/mpl-data
DEBUG:matplotlib:CONFIGDIR=/home/hayato/.config/matplotlib
DEBUG:matplotlib:matplotlib version 3.4.2
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is linux
DEBUG:matplotlib:loaded modules: ['sys', 'builtins', '_frozen_importlib', '_imp', '_warnings', '_io', 'marshal', 'posix', '_frozen_importlib_external', '_thread', '_weakref', 'time', 'zipimport', '_codecs', 'codecs', 'encodings.aliases', 'encodings', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', '_abc', 'abc', 'io', '_stat', 'stat', '_collections_abc', 'genericpath', 'posixpath', 'os.path', 'os', '_sitebuiltins', '_locale', '_bootlocale', 'types', 'importlib._bootstrap', 'importlib._bootstrap_external', 'warnings', 'importlib', 'importlib.machinery', 'importlib.abc', '_operator', 'operator', 'keyword', '_heapq', 'heapq', 'itertools', 'reprlib', '_collections', 'collections', '_fun

In [5]:
# helper functions
import functools

from acme import wrappers
import dm_env
import gym

def make_environment(level: str, evaluation: bool = False) -> dm_env.Environment:
  env = gym.make(level, full_action_space=True)

  max_episode_len = 108_000 if evaluation else 50_000

  return wrappers.wrap_all(env, [
      wrappers.GymAtariAdapter,
      functools.partial(
          wrappers.AtariWrapper,
          to_float=True,
          max_episode_len=max_episode_len,
          zero_discount_on_life_loss=True,
      ),
      wrappers.SinglePrecisionWrapper,
  ])

In [6]:
# Create a simple helper function to render a frame from the current state of
# the environment.
def render(env):
    return env.environment.render(mode='rgb_array')

def display_video(frames, filename='temp.mp4'):
  """Save and display video."""

  # Write video
  with imageio.get_writer(filename, fps=60) as video:
    for frame in frames:
      video.append_data(frame)

  # Read video and display the video
  video = open(filename, 'rb').read()
  b64_video = base64.b64encode(video)
  video_tag = ('<video  width="320" height="240" controls alt="test" '
               'src="data:video/mp4;base64,{0}">').format(b64_video.decode())

  return IPython.display.HTML(video_tag)

In [19]:
environment = make_environment('BreakoutDeterministic-v4')
environment_spec = acme.make_environment_spec(environment)
network = networks.DQNAtariNetwork(environment_spec.actions.num_values)

# Create a logger for the agent and environment loop.
# agent_logger = loggers.TerminalLogger(label='agent')
env_loop_logger = loggers.TerminalLogger(label='env_loop')

# agent = dqn.DQN(environment_spec, network, logger=agent_logger)
agent = dqn.DQN(environment_spec, network)

# Create an loop connecting this agent to the environment created above.
env_loop = acme.EnvironmentLoop(
    environment, agent, logger=env_loop_logger)
# env_loop = acme.EnvironmentLoop(environment, agent)

# Run a `num_episodes` training episodes.
# Rerun this cell until the agent has learned the given task.
# env_loop.run(num_episodes=1000)

In [20]:
# Run a `num_episodes` training episodes.
# Rerun this cell until the agent has learned the given task.
env_loop.run(num_episodes=190)

INFO:root:[Env Loop] Episode Length = 362 | Episode Return = 7.0 | Episodes = 1 | Steps = 362 | Steps Per Second = 59.213
INFO:root:[Env Loop] Episode Length = 203 | Episode Return = 3.0 | Episodes = 2 | Steps = 565 | Steps Per Second = 54.774
INFO:root:[Env Loop] Episode Length = 324 | Episode Return = 4.0 | Episodes = 3 | Steps = 889 | Steps Per Second = 58.735
INFO:tensorflow:Assets written to: /home/hayato/acme/ae827836-d837-11eb-8177-7f0a74b515f9/snapshots/network/assets
INFO:root:[Learner] Loss = 0.021 | Steps = 1 | Walltime = 0
INFO:root:[Learner] Loss = 0.187 | Steps = 8 | Walltime = 1.578
INFO:root:[Env Loop] Episode Length = 190 | Episode Return = 4.0 | Episodes = 4 | Steps = 1079 | Steps Per Second = 32.964
INFO:root:[Learner] Loss = 0.099 | Steps = 14 | Walltime = 2.683
INFO:root:[Env Loop] Episode Length = 52 | Episode Return = 2.0 | Episodes = 5 | Steps = 1131 | Steps Per Second = 50.120
INFO:root:[Learner] Loss = 0.072 | Steps = 21 | Walltime = 3.816
INFO:root:[Env Loop]

In [21]:
timestep = environment.reset()
frames = [render(environment)]

while not timestep.last():
  # Simple environment loop.
  action = agent.select_action(timestep.observation)
  timestep = environment.step(action)

  # Render the scene and add it to the frame stack.
  frames.append(render(environment))

# Save and display a video of the behaviour.
display_video(np.array(frames))

In [17]:
agent

In [10]:
from tensorflow.saved_model import load

In [22]:
agent = load('/home/hayato/acme/b8ca9590-d77d-11eb-b169-332db766104b/checkpoints/dqn_learner')

OSError: SavedModel file does not exist at: /home/hayato/acme/b8ca9590-d77d-11eb-b169-332db766104b/checkpoints/dqn_learner/{saved_model.pbtxt|saved_model.pb}

In [23]:
# agent.save('pong_model')
from acme.tf.savers import Checkpointer, Snapshotter

In [24]:
checkpointer = Checkpointer(objects_to_save={'model': network})

In [27]:
checkpointer._checkpoint_dir = '/home/hayato/acme/b8ca9590-d77d-11eb-b169-332db766104b/checkpoints/dqn_learner'

In [33]:
agent._checkpointer

AttributeError: 'Snapshot' object has no attribute '_checkpointer'

In [ ]:
from tensorflow.train import Checkpoint

In [21]:
checkpointer._checkpoint_manager._latest_checkpoint = '/home/hayato/acme/b8ca9590-d77d-11eb-b169-332db766104b/checkpoints/dqn_learner'

In [25]:
# checkpointer = Checkpointer(objects_to_save={'model': agent})
snapshot = Snapshotter(objects_to_save={"model": network})

In [26]:
snapshot.save()

INFO:tensorflow:Assets written to: /home/hayato/acme/155dbec6-d5f4-11eb-a11b-03d440faad90/snapshots/model/assets
INFO:tensorflow:Assets written to: /home/hayato/acme/155dbec6-d5f4-11eb-a11b-03d440faad90/snapshots/model/assets


True